# Preparing Data and Model for Fine-Tuning QA Pairs with Transformer Models

This notebook prepares the data and model for fine-tuning a transformer model using previously extracted question-answer (QA) pairs. It includes steps to install necessary packages, load and format the dataset, apply chat templates, and configure the model and tokenizer for efficient fine-tuning. The notebook sets up the environment and prepares the dataset, making it ready for the fine-tuning process in subsequent steps.

# Install Necessary Packages

Install all the necessary packages including Unsloth, Xformers, and others.


# Install Necessary Packages

Install all the necessary packages including Unsloth, Xformers, and others.

In [ ]:
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install datasets

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-mtq15rr1/unsloth_24c60ef3eabe4df0ba3a2b4d186568db
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-mtq15rr1/unsloth_24c60ef3eabe4df0ba3a2b4d186568db
  Resolved https://github.com/unslothai/unsloth.git to commit 27fa021a7bb959a53667dd4e7cdb9598c207aa0d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.6 MB/s eta 0:00:00
  Created wheel for unslo

# Mount Google Drive

Mount Google Drive to access and save files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load the Dataset

Load the CSV file into a pandas DataFrame and convert it to a Hugging Face Dataset.

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from unsloth.chat_templates import get_chat_template

# Load the CSV file into a pandas DataFrame
data_path = "/content/drive/MyDrive/CLT/media_qa_pairs_clean.csv"
df = pd.read_csv(data_path)

# Convert the DataFrame to a Hugging Face Dataset
dataset = Dataset.from_pandas(df)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [ ]:
dataset

Dataset({
    features: ['answer', 'question'],
    num_rows: 8402
})

# Format the Dataset

Format the dataset to the desired prompt structure.



In [ ]:
import pandas as pd
from datasets import Dataset


# Function to format the dataset
def formatting_prompts_func(examples):
    texts = [f"### Question: {q}\n ### Answer: {a}" for q, a in zip(examples["question"], examples["answer"])]
    return {"text": texts}

# Apply the formatting function to the dataset
dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/8402 [00:00<?, ? examples/s]

# Verify the Formatted Dataset



In [ ]:
dataset[0]

# Apply Chat Template to Tokenizer

Apply the chat template to the tokenizer.

In [ ]:
unsloth_template = """
{{ bos_token }}
You are a helpful assistant to the user.
{% for message in messages %}
{% if message['from'] == 'human' %}
>>> User: {{ message['value'] }}
{% elif message['from'] == 'gpt' %}
>>> Assistant: {{ message['value'] }}{{ eos_token }}
{% endif %}
{% endfor %}
{% if add_generation_prompt %}
>>> Assistant:
{% endif %}
"""

unsloth_eos_token = "eos_token"

from transformers import AutoTokenizer
from unsloth.chat_templates import get_chat_template

# Load the tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Apply the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template=(unsloth_template, unsloth_eos_token), # Provide template and EOS token
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"}, # ShareGPT style
    map_eos_token=True, # Maps  to </s> instead
)

# Verify template application
print("Template applied to tokenizer.")

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Model does not have a padding token! Will use pad_token = <|placeholder6|>.


Template applied to tokenizer.


In [ ]:
 #Function to apply chat template to the dataset
def apply_chat_template_func(examples):
    convos = [
        {"messages": [
            {"from": "human", "value": q},
            {"from": "gpt", "value": a}
        ]}
        for q, a in zip(examples["question"], examples["answer"])
    ]

    try:
        # Apply the chat template directly to the messages
        processed_convos = [
            tokenizer.apply_chat_template(convo, tokenize=True, add_special_tokens=False)
            for convo in convos
        ]
        # Extract the text from the processed conversations
        texts = ["".join(tokenizer.decode(tokens) for tokens in convo) for convo in processed_convos]
    except Exception as e:
        print(f"Error in applying chat template: {e}")
        raise e

    return {"text": texts}

# Load the tokenizer and apply the chat template
from transformers import AutoTokenizer
from unsloth.chat_templates import get_chat_template

# Define model name and load tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the template
unsloth_template = """
{{ bos_token }}
You are a helpful assistant to the user.
{% for message in messages %}
>>> {{ message['from'] }}: {{ message['value'] }}
{% endfor %}
{% if add_generation_prompt %}
>>> Assistant:
{% endif %}
"""

# Apply the chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template=(unsloth_template, "eos_token"),  # Provide template and EOS token
    mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},  # ShareGPT style
    map_eos_token=True,  # Maps to </s> instead
)

# Example conversation to verify the chat template
example_convo = {
    "messages": [
        {"from": "human", "value": "What are the advantages of the proposed adaptive-controlled AC inverter system for solar energy harvesting?"},
        {"from": "gpt", "value": "Disclosed is an adaptable DC-AC inverter system and its operation. System is suited for solar energy harvesting in grid-connected or off-grid modes of operation."}
    ]
}

# Verify the template with a single example
try:
    print("Applying chat template to a single example...")
    print(f"Example conversation structure: {example_convo}")

    # Apply chat template
    result = tokenizer.apply_chat_template(example_convo['messages'], tokenize=False, add_generation_prompt=False)

    # Print intermediate results
    print("Intermediate Result:")
    for message in example_convo["messages"]:
        print(f"{message['from']}: {message['value']}")

    print(f"Final Result: {result}")
except Exception as e:
    print(f"Error in applying chat template: {e}")
    print(f"Template: {tokenizer.chat_template}")
    raise e

# Apply the chat template function to the dataset
dataset = dataset.map(apply_chat_template_func, batched=True)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Applying chat template to a single example...
Example conversation structure: {'messages': [{'from': 'human', 'value': 'What are the advantages of the proposed adaptive-controlled AC inverter system for solar energy harvesting?'}, {'from': 'gpt', 'value': 'Disclosed is an adaptable DC-AC inverter system and its operation. System is suited for solar energy harvesting in grid-connected or off-grid modes of operation.'}]}
Intermediate Result:
human: What are the advantages of the proposed adaptive-controlled AC inverter system for solar energy harvesting?
gpt: Disclosed is an adaptable DC-AC inverter system and its operation. System is suited for solar energy harvesting in grid-connected or off-grid modes of operation.
Final Result: 
<s>
You are a helpful assistant to the user.
>>> human: What are the advantages of the proposed adaptive-controlled AC inverter system for solar energy harvesting?
>>> gpt: Disclosed is an adaptable DC-AC inverter system and its operation. System is suited fo

Map:   0%|          | 0/8402 [00:00<?, ? examples/s]

# Apply Chat Template to Dataset

Function to apply the chat template to the dataset and process it.

In [ ]:
# Function to apply chat template to the dataset
def apply_chat_template_func(examples):
    convos = [
        {"messages": [
            {"from": "human", "value": q},
            {"from": "gpt", "value": a}
        ]}
        for q, a in zip(examples["question"], examples["answer"])
    ]

    try:
        # Apply the chat template directly to the messages
        processed_convos = [
            tokenizer.apply_chat_template(convo['messages'], tokenize=True, add_special_tokens=False)
            for convo in convos
        ]

        # Debugging: Print the structure of processed_convos
        print("Processed Conversations (First Example):", processed_convos[0])

        # Extract the text from the processed conversations
        texts = [tokenizer.decode(convo) for convo in processed_convos]
    except Exception as e:
        print(f"Error in applying chat template: {e}")
        raise e

    return {"text": texts}

# Apply the chat template function to the dataset
dataset = dataset.map(apply_chat_template_func, batched=True)

# Check the first few rows to ensure it processed correctly
print(dataset["text"][:5])


Map:   0%|          | 0/8402 [00:00<?, ? examples/s]

Processed Conversations (First Example): [29871, 13, 1, 29871, 13, 3492, 526, 263, 8444, 20255, 304, 278, 1404, 29889, 13, 6778, 29958, 5199, 29901, 1724, 526, 278, 1820, 16650, 583, 660, 14873, 5879, 12154, 398, 313, 29984, 29925, 29897, 338, 16049, 304, 946, 3663, 3598, 5700, 7933, 8697, 10489, 953, 6847, 322, 10032, 10489, 1652, 4362, 26171, 297, 967, 1963, 559, 29871, 29906, 365, 9312, 13184, 322, 701, 5461, 23330, 29973, 13, 6778, 29958, 330, 415, 29901, 660, 14873, 5879, 12154, 398, 313, 660, 29925, 29897, 338, 3646, 292, 946, 3663, 573, 5700, 29879, 297, 967, 7933, 8697, 10489, 953, 6847, 408, 372, 10223, 267, 304, 6826, 1963, 559, 29871, 29906, 310, 967, 20458, 29871, 29946, 29947, 7284, 15243, 639, 1629, 365, 9312, 13184, 29889, 450, 5001, 338, 884, 12242, 292, 304, 10032, 10489, 1652, 4362, 26171, 4822, 967, 701, 5461, 23330, 29889, 13]
Processed Conversations (First Example): [29871, 13, 1, 29871, 13, 3492, 526, 263, 8444, 20255, 304, 278, 1404, 29889, 13, 6778, 29958, 5199,

# Verify the Processed Dataset

Check the first few rows to ensure it processed correctly.

In [ ]:
# Check the first few rows to ensure it processed correctly
for i in range(5):
    print(f"Processed Example {i}:")
    print(dataset["text"][i])
    print("\n")

# Save the processed dataset to a CSV file
processed_dataset_path = "/content/drive/MyDrive/CLT/media_qa_pairs_processed.csv"
dataset.to_csv(processed_dataset_path, index=False)

print(f"Processed dataset saved to {processed_dataset_path}")

Processed Example 0:

<s> 
You are a helpful assistant to the user.
>>> human: What are the key strategies Qatar Petroleum (QP) is implementing to aggressively cut greenhouse gas emissions and reduce gas flaring intensity in its Phase 2 LNG expansion and upstream facilities?
>>> gpt: Qatar Petroleum ( QP) is targeting aggressive cuts in its greenhouse gas emissions as it prepares to launch Phase 2 of its planned 48 million ton per year LNG expansion. The company is also aiming to reduce gas flaring intensity across its upstream facilities.



Processed Example 1:

<s> 
You are a helpful assistant to the user.
>>> human: What is the significance of Kakrapar-3 being the first of India's 700 MW indigenously developed pressurized heavy water reactors (PHWRs), and how many more units of the same design are planned to follow?
>>> gpt: Kakrapar-3 is the first of India's 700 megawatt indigenously developed pressurized heavy water reactors ( PHWRs) to reach this milestone. 15 more units of the 

Creating CSV from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Processed dataset saved to /content/drive/MyDrive/CLT/media_qa_pairs_processed.csv


# Load and Quantize the Model

Load the pre-trained model and quantize it if necessary.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Configure PEFT (Parameter-Efficient Fine-Tuning)

Apply PEFT to the model.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Configure the Trainer

Configure the trainer for fine-tuning.



In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/8402 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


# Show Current Memory Stats

Display current memory statistics.

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.283 GB of memory reserved.


# Train the Model

Train the model with the configured trainer.

In [ ]:
trainer_stats = trainer.train()

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "phi-3", # Supports zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, unsloth
    mapping = {"role" : "from", "content" : "value", "user" : "human", "assistant" : "gpt"}, # ShareGPT style
)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"from": "human", "value": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)